# word2vec&CNN

it is an example for sentiment analysis using word2vec and cnn netural network.

In [1]:
from gensim.models import word2vec
from sklearn import svm
import numpy as np
import pandas as pd
import jieba
olderr = np.seterr(all='ignore')

C:\Users\ty020\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
f = open("D:\\杂七杂八课程\\CUMCM-template-note\\NLP\\doubanMoviereview--master\\scrapy_data\\data_long\\三傻大闹宝莱坞.csv")
raw_data = pd.read_csv(f,usecols=['评论内容', '推荐星数'])
raw_data
raw_labels = raw_data["推荐星数"].tolist()
y_labels = []
for label in raw_labels:
    if label == "还行" or label == "很差" or label == "较差":
        y_labels.append(0)
    else:
        y_labels.append(1)
x_comments = raw_data["评论内容"].tolist()
positive_comments = []
negative_comments = []
tmp_comments = []
for label,comment in zip(y_labels, x_comments):
    if(label == 1):
        positive_comments += jieba.cut(comment.strip(), cut_all=False)
    else:
        negative_comments += jieba.cut(comment.strip(), cut_all=False)
for i in range(len(positive_comments)):
    if positive_comments[i] not in "、，“”。？！；：...1234567890‘’-《》 ":
        tmp_comments.append(positive_comments[i])
positive_comments = tmp_comments

tmp_comments = []
for i in range(len(negative_comments)):
    if negative_comments[i] not in "、，“”。？！；：...1234567890‘’-《》 ":
        tmp_comments.append(negative_comments[i])
negative_comments = tmp_comments

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ty020\AppData\Local\Temp\jieba.cache
Loading model cost 0.692 seconds.
Prefix dict has been built succesfully.


In [3]:
comments = positive_comments + negative_comments
with open("D:\\杂七杂八课程\\CUMCM-template-note\\NLP\\doubanMoviereview--master\\scrapy_data\\data_long\\test.txt","w",encoding='utf-8') as fw:
    for sentence in comments:
        data=sentence.strip()  
        if len(data)!=0:  
            fw.write(data)
            fw.write("\n") 

sentences=word2vec.Text8Corpus('D:\\杂七杂八课程\\CUMCM-template-note\\NLP\\doubanMoviereview--master\\scrapy_data\\data_long\\test.txt')
model=word2vec.Word2Vec(sentences, size=40)

comments_vec = []
for i in range(len(y_labels)):
    result = list(jieba.cut(x_comments[i].strip(), cut_all=False))
    tmp = []
    for j in range(len(result)):
        if result[j] not in "、，“”。？！；：...1234567890‘’-《》 ":
            tmp.append(result[j])
    result = tmp
    single_vec = np.zeros(40)
    wrong = 0;
    for word in result:
        try:
            single_vec += np.array(model.wv[word])
        except KeyError:
            wrong += 1
            continue
    single_vec /= (len(result) - wrong+1)
    comments_vec.append(single_vec)
comments_vec

[array([ 0.0723438 , -0.24510796,  0.19041228, -0.08720314,  0.04610388,
        -0.06787333, -0.00181929,  0.03996007,  0.0089769 , -0.04265409,
         0.10439957,  0.05459479,  0.0115734 ,  0.0073037 , -0.0159374 ,
        -0.07220597, -0.11026239, -0.21250725, -0.09698652, -0.0978028 ,
        -0.25711356, -0.10246138,  0.00490924,  0.09522458,  0.0510768 ,
         0.01377082, -0.17629891, -0.35991927, -0.03052792,  0.01297298,
         0.03473044,  0.00155302,  0.02372904, -0.18373304,  0.14465343,
        -0.22549266,  0.0524619 ,  0.16602546, -0.03035477, -0.14716395]),
 array([ 0.09077419, -0.30582669,  0.23965729, -0.1092158 ,  0.06140936,
        -0.08200044, -0.00404026,  0.04997246,  0.0113607 , -0.05385376,
         0.13032038,  0.06637435,  0.01443264,  0.00956458, -0.02078373,
        -0.0905612 , -0.13872023, -0.266467  , -0.12205024, -0.12296195,
        -0.32445633, -0.1292374 ,  0.00600685,  0.11876498,  0.06382659,
         0.01525332, -0.22296456, -0.45220423, -0

In [4]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=20)
comments_vec = pca.fit_transform(comments_vec)
comments_vec.shape

(3609, 20)

In [6]:
#split train and split
from sklearn.model_selection import train_test_split
train_comments, test_comments, train_labels, test_labels = train_test_split(comments_vec, y_labels, test_size=0.33, random_state=42)

## To train in CNN

In [24]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
with tf.device("/gpu:0"):
    model = tf.keras.Sequential([
    keras.layers.Dense(64, input_shape=(20,)),
    keras.layers.Dropout(rate=0.3,seed=1),
    keras.layers.Dense(32,activation=tf.nn.softmax,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01)),
    keras.layers.Dense(2, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_comments, train_labels, epochs=20)
    
    test_loss, test_acc = model.evaluate(test_comments, test_labels)

Using TensorFlow backend.


Epoch 1/20
2418/2418 [==============================] - 1s 274us/step - loss: 1.2219 - acc: 0.8941
Epoch 2/20
2418/2418 [==============================] - 0s 70us/step - loss: 0.9290 - acc: 0.9376
Epoch 3/20
2418/2418 [==============================] - 0s 75us/step - loss: 0.7561 - acc: 0.9376
Epoch 4/20
2418/2418 [==============================] - 0s 77us/step - loss: 0.6343 - acc: 0.9376
Epoch 5/20
2418/2418 [==============================] - 0s 73us/step - loss: 0.6023 - acc: 0.9376
Epoch 6/20
2418/2418 [==============================] - 0s 72us/step - loss: 0.5872 - acc: 0.9376
Epoch 7/20
2418/2418 [==============================] - 0s 70us/step - loss: 0.5796 - acc: 0.9376
Epoch 8/20
2418/2418 [==============================] - 0s 71us/step - loss: 0.5741 - acc: 0.9376
Epoch 9/20
2418/2418 [==============================] - 0s 71us/step - loss: 0.5709 - acc: 0.9376
Epoch 10/20
2418/2418 [==============================] - 0s 71us/step - loss: 0.5682 - acc: 0.9376
Epoch 11/20
2418/2

In [25]:
test_loss, test_acc

(0.563364338526998, 0.9353484466834593)